# Fine-Tune Whisper For Multilingual ASR with 🤗 Transformers

## Prepare Environment

We can verify that we've been assigned a GPU and view its specifications:

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Dec  7 22:23:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Next, we need to update the Unix package `ffmpeg` to version 4:

In [ ]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

Repository: 'deb https://ppa.launchpadcontent.net/jonathonf/ffmpeg-4/ubuntu/ jammy main'
Description:
Backport of FFmpeg 4 and associated libraries. Now includes AOM/AV1 support!

FDK AAC is not compatible with GPL and FFmpeg can't be redistributed with it included. Please don't ask for it to be added to this public PPA.

---

PPA supporters:

BigBlueButton (https://bigbluebutton.org)

---

Donate to FFMPEG: https://ffmpeg.org/donations.html
Donate to Debian: https://www.debian.org/donations
Donate to this PPA: https://ko-fi.com/jonathonf
More info: https://launchpad.net/~jonathonf/+archive/ubuntu/ffmpeg-4
Adding repository.
Adding deb entry to /etc/apt/sources.list.d/jonathonf-ubuntu-ffmpeg-4-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/jonathonf-ubuntu-ffmpeg-4-jammy.list
Adding key to /etc/apt/trusted.gpg.d/jonathonf-ubuntu-ffmpeg-4.gpg with fingerprint 4AB0F789CBA31744CC7DA76A8CF63AD3F06FC659
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ In

We'll employ several popular Python packages to fine-tune the Whisper model.
We'll use `datasets` to download and prepare our training data and
`transformers` to load and train our Whisper model. We'll also require
the `soundfile` package to pre-process audio files, `evaluate` and `jiwer` to
assess the performance of our model. Finally, we'll
use `gradio` to build a flashy demo of our fine-tuned model.

In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install hopsworks

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-dnqkxu19
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-dnqkxu19
  Resolved https://github.com/huggingface/transformers to commit 0ea42ef0f9f71deba7775ead33afa0e493823d60
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8178366 sha256=6a0aca021734692efe7eebaa1dd290d0393f6f8ef2d632faf82422f0239c6095
  Stored in directory: /tmp/pip-ephem-wheel-cache-4b8onp88/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Load Dataset

Using 🤗 Datasets, downloading and preparing data is extremely simple.
We can download and prepare the Common Voice splits in just one line of code.

First, ensure you have accepted the terms of use on the Hugging Face Hub: [mozilla-foundation/common_voice_11_0](https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0). Once you have accepted the terms, you will have full access to the dataset and be able to download the data locally.

The bengali dataset from mozilla-foundation is further split into a smaller training and a smaller test set as the size was too large for Google Drive. 

In [ ]:
from datasets import load_dataset, DatasetDict
import random

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "bn", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "bn", split="test", use_auth_token=True)

# Select a random 15% of the train split
percentage_train = 0.15
train_size = int(len(common_voice["train"]) * percentage_train)
random_train_indices = random.sample(range(len(common_voice["train"])), train_size)
common_voice["train"] = common_voice["train"].select(random_train_indices)

# Select a random 3% of the test split
percentage_test = 0.3
test_size = int(len(common_voice["test"]) * percentage_test)
random_test_indices = random.sample(range(len(common_voice["test"])), test_size)
common_voice["test"] = common_voice["test"].select(random_test_indices)

print(common_voice)

You can remove this warning by passing 'token=<use_auth_token>' instead.


Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]



Reading metadata...: 0it [00:00, ?it/s]

Reading metadata...: 16777it [00:00, 92370.25it/s]


...

Generating other split: 0 examples [00:00, ? examples/s]



Reading metadata...: 0it [00:00, ?it/s]

Reading metadata...: 6587it [00:00, 65860.48it/s]

Reading metadata...: 14596it [00:00, 74222.21it/s]

Reading metadata...: 22519it [00:00, 76504.78it/s]

Reading metadata...: 30380it [00:00, 77334.04it/s]

Reading metadata...: 38114it [00:00, 76666.24it/s]

Reading metadata...: 45782it [00:00, 76565.28it/s]

Reading metadata...: 55171it [00:00, 82199.29it/s]

Reading metadata...: 66999it [00:00, 93645.79it/s]

Reading metadata...: 78907it [00:00, 101575.53it/s]

Reading metadata...: 89330it [00:01, 102378.76it/s]

Reading metadata...: 100972it [00:01, 106669.50it/s]

Reading metadata...: 112210it [00:01, 108403.12it/s]

Reading metadata...: 123093it [00:01, 108529.49it/s]

Reading metadata...: 133948it [00:01, 107203.58it/s]

Reading metadata...: 146841it [00:01, 113691.98it/s]

Reading metadata...: 159882it [00:01, 118690.51it/s]

Reading metadata...: 171761it [00:01, 113210.25it/s]

Reading metadata...: 183141it [00:01, 108424.38it/s]

Read

Generating invalidated split: 0 examples [00:00, ? examples/s]



Reading metadata...: 6447it [00:00, 92811.66it/s]


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 3769
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 2505
    })
})


In [ ]:
#Mount the Google Drive

from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')




Mounted at /content/drive


In [ ]:
#Save the Dataset for Common Voice Bengali in Google Drive, NOT NEEDED

#common_voice.save_to_disk("/content/drive/MyDrive/voice_dataset")

Saving the dataset (0/2 shards):   0%|          | 0/25130 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/8353 [00:00<?, ? examples/s]

Most ASR datasets only provide input audio samples (`audio`) and the
corresponding transcribed text (`sentence`). Common Voice contains additional
metadata information, such as `accent` and `locale`, which we can disregard for ASR.
Keeping the notebook as general as possible, we only consider the input audio and
transcribed text for fine-tuning, discarding the additional metadata information:

In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

#Have killed the excess features

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 3769
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 2505
    })
})


## Prepare Feature Extractor, Tokenizer and Data

The ASR pipeline can be de-composed into three stages:
1) A feature extractor which pre-processes the raw audio-inputs
2) The model which performs the sequence-to-sequence mapping
3) A tokenizer which post-processes the model outputs to text format

In 🤗 Transformers, the Whisper model has an associated feature extractor and tokenizer,
called [WhisperFeatureExtractor](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperFeatureExtractor)
and [WhisperTokenizer](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperTokenizer)
respectively.

We'll go through details for setting-up the feature extractor and tokenizer one-by-one!

### Load WhisperFeatureExtractor

The Whisper feature extractor performs two operations:
1. Pads / truncates the audio inputs to 30s: any audio inputs shorter than 30s are padded to 30s with silence (zeros), and those longer that 30s are truncated to 30s
2. Converts the audio inputs to _log-Mel spectrogram_ input features, a visual representation of the audio and the form of the input expected by the Whisper model

<figure>
<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/spectrogram.jpg" alt="Trulli" style="width:100%">
<figcaption align = "center"><b>Figure 2:</b> Conversion of sampled audio array to log-Mel spectrogram.
Left: sampled 1-dimensional audio signal. Right: corresponding log-Mel spectrogram. Figure source:
<a href="https://ai.googleblog.com/2019/04/specaugment-new-data-augmentation.html">Google SpecAugment Blog</a>.
</figcaption>

We'll load the feature extractor from the pre-trained checkpoint with the default values:

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")             # Bigger model could be used

### Load WhisperTokenizer

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Bengali", task="transcribe")

### Combine To Create A WhisperProcessor

To simplify using the feature extractor and tokenizer, we can _wrap_
both into a single `WhisperProcessor` class. This processor object
inherits from the `WhisperFeatureExtractor` and `WhisperProcessor`,
and can be used on the audio inputs and model predictions as required.
In doing so, we only need to keep track of two objects during training:
the `processor` and the `model`:

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Bengali", task="transcribe")

### Prepare Data

Let's print the first example of the Common Voice dataset to see
what form the data is in:

In [ ]:
print(common_voice["train"][0])

NameError: ignored

Since
our input audio is sampled at 48kHz, we need to _downsample_ it to
16kHz prior to passing it to the Whisper feature extractor, 16kHz being the sampling rate expected by the Whisper model.

We'll set the audio inputs to the correct sampling rate using dataset's
[`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column)
method. This operation does not change the audio in-place,
but rather signals to `datasets` to resample audio samples _on the fly_ the
first time that they are loaded:

In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

ValueError: ignored

Re-loading the first audio sample in the Common Voice dataset will resample
it to the desired sampling rate:

In [ ]:
#Notice the change in the 'sampling_rate' now

print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/3176f7d070e0a8030b3742168b5aa12aa22dad56beb5338e7431487ab248be83/bn_train_0/common_voice_bn_31610365.mp3', 'array': array([-1.72803993e-11,  4.54747351e-13, -1.27329258e-11, ...,
        1.79178869e-06,  4.56716066e-07,  1.21163100e-06]), 'sampling_rate': 16000}, 'sentence': 'প্রদেশটি সপ্তম শতাব্দীর মুসলিম আরব অভিযান পর্যন্ত টিকে ছিল।'}


Now we can write a function to prepare our data ready for the model:
1. We load and resample the audio data by calling `batch["audio"]`. As explained above, 🤗 Datasets performs any necessary resampling operations on the fly.
2. We use the feature extractor to compute the log-Mel spectrogram input features from our 1-dimensional audio array.
3. We encode the transcriptions to label ids through the use of the tokenizer.

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

We can apply the data preparation function to all of our training examples using dataset's `.map` method. The argument `num_proc` specifies how many CPU cores to use. Setting `num_proc` > 1 will enable multiprocessing. If the `.map` method hangs with multiprocessing, set `num_proc=1` and process the dataset sequentially.

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

Map (num_proc=2):   0%|          | 0/3769 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2505 [00:00<?, ? examples/s]

In [ ]:
help(common_voice)

Help on DatasetDict in module datasets.dataset_dict object:

class DatasetDict(builtins.dict)
 |  A dictionary (dict of str: datasets.Dataset) with dataset transforms methods (map, filter, etc.)
 |  
 |  Method resolution order:
 |      DatasetDict
 |      builtins.dict
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, k) -> datasets.arrow_dataset.Dataset
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  align_labels_with_mapping(self, label2id: Dict, label_column: str) -> 'DatasetDict'
 |      Align the dataset's label ID and label name mapping to match an input `label2id` mapping.
 |      This is useful when you want to ensure that a model's predicted labels are aligned with the dataset.
 |      The alignment in done using the lowercase label names.
 |      
 |      Args:
 |          label2id (`dict`):
 |              The label name to ID mapping to align the dataset with.
 |          label_column (`str`):
 

In [ ]:
# Saving to my Google Drive also

#common_voice.save_to_disk("common_voice") better where I can see it so avoiding this

common_voice.save_to_disk("/content/drive/MyDrive/common_voice")

Saving the dataset (0/8 shards):   0%|          | 0/3769 [00:00<?, ? examples/s]

Saving the dataset (0/5 shards):   0%|          | 0/2505 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset, DatasetDict

cc = DatasetDict.load_from_disk("/content/drive/MyDrive/common_voice")

In [ ]:
import os
print(os.getcwd())
print(os.listdir("/content/drive/MyDrive/common_voice/"))
print(os.listdir("/content/drive/MyDrive/common_voice/train"))
print(os.listdir("/content/drive/MyDrive/common_voice/test"))

/content
['dataset_dict.json', 'test', 'train', '.ipynb_checkpoints']
['data-00000-of-00008.arrow', 'data-00001-of-00008.arrow', 'data-00002-of-00008.arrow', 'data-00003-of-00008.arrow', 'data-00004-of-00008.arrow', 'data-00005-of-00008.arrow', 'data-00006-of-00008.arrow', 'data-00007-of-00008.arrow', 'dataset_info.json', 'state.json']
['data-00000-of-00005.arrow', 'data-00001-of-00005.arrow', 'data-00002-of-00005.arrow', 'data-00003-of-00005.arrow', 'data-00004-of-00005.arrow', 'state.json', 'dataset_info.json']


In [ ]:
import os

def get_dir_size(path='/content/drive/MyDrive/common_voice/train'):
    total = 0
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_file():
                total += entry.stat().st_size
            elif entry.is_dir():
                total += get_dir_size(entry.path)
    return total
sz = get_dir_size(path="/content/drive/MyDrive/common_voice/")
print(sz)

6031575666


In [ ]:
# Save your dataset to google drive
# This is untested code - just a suggestion
#common_voice.save_to_disk(F"/content/gdrive/My Drive/common_voice/")

In [ ]:
#os.mkdir("/content/gdrive/My Drive/common_voice")

In [ ]:
# Why even do this... ?

#import shutil
#shutil.move("/content/gdrive/MyDrive/dataset_dict.json", "/content/gdrive/MyDrive/common_voice")

In [ ]:
print(os.listdir(F"/content/drive/MyDrive/common_voice"))

['dataset_dict.json', 'train', 'test']


In [ ]:
cc2 = DatasetDict.load_from_disk("/content/drive/MyDrive/common_voice")

In [ ]:
cc2   # up to here perfect

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 3769
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 2505
    })
})

In [ ]:
#SKIPPING THIS PART FOR NOW, use the files directly from google drive

# Alternatively, you can upload it to hopsworks
dataset_api = project.get_dataset_api()

# IN HOPSWORKS GO TO SETTINGS, THEN FILE BROWSER THEN CREATE DATASET "/common_voice"
# TODO - create the directories in Hopsworks in the file browser inside some existing dataset
# common_voice/train
# common_voice/test

In [ ]:
# TODO - upload all of the local files to the common_voice directory you created in Hopsworks

uploaded_file_path = dataset_api.upload(
    local_path = "/content/drive/MyDrive/common_voice/dataset_dict.json",
    upload_path = "/common_voice/", overwrite=True)

Uploading: 0.000%|          | 0/29 elapsed<00:00 remaining<?

In [ ]:
uploaded_file_path = dataset_api.upload(
    local_path = "/content/drive/MyDrive/common_voice/train/state.json",
    upload_path = "/common_voice/train/")

Uploading: 0.000%|          | 0/660 elapsed<00:00 remaining<?

In [ ]:
uploaded_file_path = dataset_api.upload(
    local_path = "/content/drive/MyDrive/common_voice/test/state.json",
    upload_path = "/common_voice/test/")

Uploading: 0.000%|          | 0/483 elapsed<00:00 remaining<?

In [ ]:
import os

local_path = "/content/drive/MyDrive/common_voice/test"
all_files = os.listdir(local_path)
arrow_files = [file for file in all_files if file.endswith(".arrow")]

for arrow_file in arrow_files:
    file_path = os.path.join(local_path, arrow_file)

    # Your code to process each .arrow file goes here
    print(f"Processing {file_path}")
    uploaded_file_path = dataset_api.upload(
      local_path = file_path,
      upload_path = "/common_voice/test/")

#-------------------------------------------------------------
#uploaded_file_path = dataset_api.upload(
#    local_path = "/content/drive/MyDrive/common_voice/test/dataset.arrow",
#    upload_path = "/common_voice/test/")

Processing /content/drive/MyDrive/common_voice/test/data-00000-of-00005.arrow


Uploading: 0.000%|          | 0/481645016 elapsed<00:00 remaining<?

Processing /content/drive/MyDrive/common_voice/test/data-00001-of-00005.arrow


Uploading: 0.000%|          | 0/481628176 elapsed<00:00 remaining<?

Processing /content/drive/MyDrive/common_voice/test/data-00002-of-00005.arrow


Uploading: 0.000%|          | 0/481636456 elapsed<00:00 remaining<?

RestAPIError: ignored

In [ ]:
import os

local_path = "/content/drive/MyDrive/common_voice/train"
all_files = os.listdir(local_path)
arrow_files = [file for file in all_files if file.endswith(".arrow")]

for arrow_file in arrow_files:
    file_path = os.path.join(local_path, arrow_file)

    # Your code to process each .arrow file goes here
    print(f"Processing {file_path}")
    uploaded_file_path = dataset_api.upload(
      local_path = file_path,
      upload_path = "/common_voice/train/")

In [ ]:
from datasets import DatasetDict

common_voice = DatasetDict.load_from_disk("/content/drive/MyDrive/common_voice")

## Training and Evaluation

Now that we've prepared our data, we're ready to dive into the training pipeline.
The [🤗 Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer)
will do much of the heavy lifting for us. All we have to do is:

- Define a data collator: the data collator takes our pre-processed data and prepares PyTorch tensors ready for the model.

- Evaluation metrics: during evaluation, we want to evaluate the model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric. We need to define a `compute_metrics` function that handles this computation.

- Load a pre-trained checkpoint: we need to load a pre-trained checkpoint and configure it correctly for training.

- Define the training configuration: this will be used by the 🤗 Trainer to define the training schedule.

Once we've fine-tuned the model, we will evaluate it on the test data to verify that we have correctly trained it
to transcribe speech in Hindi.

### Define a Data Collator

The data collator for a sequence-to-sequence speech model is unique in the sense that it
treats the `input_features` and `labels` independently: the  `input_features` must be
handled by the feature extractor and the `labels` by the tokenizer.

The `input_features` are already padded to 30s and converted to a log-Mel spectrogram
of fixed dimension by action of the feature extractor, so all we have to do is convert the `input_features`
to batched PyTorch tensors. We do this using the feature extractor's `.pad` method with `return_tensors=pt`.

The `labels` on the other hand are un-padded. We first pad the sequences
to the maximum length in the batch using the tokenizer's `.pad` method. The padding tokens
are then replaced by `-100` so that these tokens are **not** taken into account when
computing the loss. We then cut the BOS token from the start of the label sequence as we
append it later during training.

We can leverage the `WhisperProcessor` we defined earlier to perform both the
feature extractor and the tokenizer operations:

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Let's initialise the data collator we've just defined:

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### Evaluation Metrics

We'll use the word error rate (WER) metric, the 'de-facto' metric for assessing
ASR systems. For more information, refer to the WER [docs](https://huggingface.co/metrics/wer). We'll load the WER metric from 🤗 Evaluate:

In [ ]:
import evaluate

metric = evaluate.load("wer")

We then simply have to define a function that takes our model
predictions and returns the WER metric. This function, called
`compute_metrics`, first replaces `-100` with the `pad_token_id`
in the `label_ids` (undoing the step we applied in the
data collator to ignore padded tokens correctly in the loss).
It then decodes the predicted and label ids to strings. Finally,
it computes the WER between the predictions and reference labels:

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### Load a Pre-Trained Checkpoint

Now let's load the pre-trained Whisper `small` checkpoint. Again, this
is trivial through use of 🤗 Transformers!

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

Override generation arguments - no tokens are forced as decoder outputs (see [`forced_decoder_ids`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.forced_decoder_ids)), no tokens are suppressed during generation (see [`suppress_tokens`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.suppress_tokens)):

In [ ]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

### Define the Training Configuration

In the final step, we define all the parameters related to training. For more detail on the training arguments, refer to the Seq2SeqTrainingArguments [docs](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments).

In [ ]:
from transformers import Seq2SeqTrainingArguments

! pip install -U accelerate
! pip install -U transformers

training_args = Seq2SeqTrainingArguments(
    num_train_epochs=1,
    #tash
    output_dir="/content/drive/MyDrive/final_model",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=250,
    max_steps=1000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)


"""
training_args = Seq2SeqTrainingArguments(
    num_train_epochs=1,
    #michael jackson
    output_dir="/content/drive/MyDrive/testing_model",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)
"""

'\ntraining_args = Seq2SeqTrainingArguments(\n    num_train_epochs=1,\n    #michael jackson\n    output_dir="/content/drive/MyDrive/testing_model",  # change to a repo name of your choice\n    per_device_train_batch_size=16,\n    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size\n    learning_rate=1e-5,\n    warmup_steps=500,\n    max_steps=4000,\n    gradient_checkpointing=True,\n    fp16=True,\n    evaluation_strategy="steps",\n    per_device_eval_batch_size=8,\n    predict_with_generate=True,\n    generation_max_length=225,\n    save_steps=500,\n    eval_steps=500,\n    logging_steps=25,\n    report_to=["tensorboard"],\n    load_best_model_at_end=True,\n    metric_for_best_model="wer",\n    greater_is_better=False,\n    push_to_hub=True,\n)\n'

**Note**: if one does not want to upload the model checkpoints to the Hub,
set `push_to_hub=False`.

We can forward the training arguments to the 🤗 Trainer along with our model,
dataset, data collator and `compute_metrics` function:

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

We'll save the processor object once before starting training. Since the processor is not trainable, it won't change over the course of training:

In [ ]:
processor.save_pretrained(training_args.output_dir)

### Training

Training will take approximately 5-10 hours depending on your GPU or the one
allocated to this Google Colab. If using this Google Colab directly to
fine-tune a Whisper model, you should make sure that training isn't
interrupted due to inactivity. A simple workaround to prevent this is
to paste the following code into the console of this tab (_right mouse click_
-> _inspect_ -> _Console tab_ -> _insert code_).

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton, 60000);
```

The peak GPU memory for the given training configuration is approximately 15.8GB.
Depending on the GPU allocated to the Google Colab, it is possible that you will encounter a CUDA `"out-of-memory"` error when you launch training.
In this case, you can reduce the `per_device_train_batch_size` incrementally by factors of 2
and employ [`gradient_accumulation_steps`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments.gradient_accumulation_steps)
to compensate.

To launch training, simply execute:

In [ ]:
trainer.train()
#trainer.train(resume_from_checkpoint=True)

`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
500,0.128200,0.223264,57.797886
1000,0.051000,0.208389,52.477487


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


TrainOutput(global_step=1000, training_loss=0.33534482395648957, metrics={'train_runtime': 12875.104, 'train_samples_per_second': 1.243, 'train_steps_per_second': 0.078, 'total_flos': 4.60928601391104e+18, 'train_loss': 0.33534482395648957, 'epoch': 4.24})

Our best WER is 32.0% - not bad for 8h of training data! We can submit our checkpoint to the [`hf-speech-bench`](https://huggingface.co/spaces/huggingface/hf-speech-bench) on push by setting the appropriate key-word arguments (kwargs):

In [ ]:
kwargs = {
    "dataset_tags": ["mozilla-foundation", "common_voice_11_0"],
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": {"config": "hi", "split": "test"},
    "language": "bn",
    "model_name": "Whisper Small Hi - bn",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": ["hf-asr-leaderboard"],
}


The training results can now be uploaded to the Hub. To do so, execute the `push_to_hub` command and save the preprocessor object we created:

In [ ]:
trainer.push_to_hub(**kwargs)

BadRequestError: ignored